# Entorno Remoto de Compilación (MambaVision) 🚀

Este notebook se ejecuta enteramente en los servidores de Google Colab conectado desde tu VS Code.
Sigue los pasos secuenciales para descargar tu código desde GitHub y compilar el Kernel Custom de C++ contra los headers de TensorFlow provistos por Google.

In [1]:
import os
import sys

# ==========================================
# CONFIGURACIÓN DE TU REPOSITORIO DE GITHUB
# ==========================================
GITHUB_REPO = "https://github.com/rdromer2/mambavision_tf_kernel.git"
REPO_NAME = "mambavision_tf_kernel"
PROJECT_PATH = f"/content/{REPO_NAME}"

# 1. Clonar o actualizar el repositorio
if not os.path.exists(PROJECT_PATH):
    print(f"Clonando el repositorio {GITHUB_REPO}...")
    os.system(f'git clone {GITHUB_REPO} {PROJECT_PATH}')
else:
    print("El repositorio ya existe. Actualizando con git pull...")
    os.system(f'cd {PROJECT_PATH} && git pull')

# 2. Añadir al sys.path
sys.path.append(PROJECT_PATH)

# 3. Cambiar nuestro directorio activo al proyecto
os.chdir(PROJECT_PATH)
print(f"\nDirectorio de trabajo actual: {os.getcwd()}")

Clonando el repositorio https://github.com/rdromer2/mambavision_tf_kernel.git...

Directorio de trabajo actual: /content/mambavision_tf_kernel


## 🛠️ Compilación C++ delegada en CMake
Delegamos integramente el proceso de construcción en CMake en la subcarpeta `build/`, lo que garantiza mayor mantenibilidad y asegura que las aserciones formales y el fix de ABI establecidos en `CMakeLists.txt` surtan efecto sistemáticamente.

In [ ]:
import os

print("Configurando y Compilando el Kernel C++ con CMake...")
!mkdir -p {PROJECT_PATH}/build
%cd {PROJECT_PATH}/build
!cmake ..
!make -j
%cd {PROJECT_PATH}

print("\n✅ Compilación finalizada.")

## 🧪 Prueba Inicial del Kernel (El Centinela de Memoria)
**Detalle Técnico**: Mantenemos el hack de `ctypes.RTLD_GLOBAL` vital porque seguimos utilizando Colab y los binarios de TF no están compilados estáticamente.
Esta prueba evaluará un flujo correcto (3D) y un caso de fallo intencionado (2D) para verificar la intercepción del centinela de memoria.

In [ ]:
import ctypes
import tensorflow as tf
import glob
import os

# === MANTENEMOS EL FIX DE ABI PARA TENSORFLOW ===
# Cargar libtensorflow_framework.so con RTLD_GLOBAL para que sus símbolos
# estén disponibles cuando se cargue nuestra librería en runtime.
tf_lib_dir = tf.sysconfig.get_lib()
fw_libs = glob.glob(os.path.join(tf_lib_dir, 'libtensorflow_framework.so*'))
if fw_libs:
    lib_path = sorted(fw_libs)[-1]
    print(f'Precargando RTLD_GLOBAL: {lib_path}')
    ctypes.CDLL(lib_path, ctypes.RTLD_GLOBAL)
else:
    print('⚠️ No se encontró libtensorflow_framework.so')

try:
    # 1. Cargar el nuevo binario
    mamba_module = tf.load_op_library(f'{PROJECT_PATH}/build/libmamba_kernel.so')
    
    # 2. Prueba de Éxito: Crear un tensor 3D aleatorio (Batch=2, Seq=128, Dim=64)
    tensor_valido = tf.random.normal([2, 128, 64])
    resultado = mamba_module.mamba_selective_scan(tensor_valido)
    print("\n✅ Prueba 3D exitosa. Forma de salida:", resultado.shape)

    # 3. Prueba de Fallo (El Centinela en acción): Enviar un tensor 2D
    tensor_invalido = tf.random.normal([128, 64])
    print("\nIntentando procesar tensor 2D...")
    mamba_module.mamba_selective_scan(tensor_invalido)

except Exception as e:
    print("\n🛡️ Validación interceptada correctamente por C++:")
    print("Error:", e)
